In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


### Load one DF

In [2]:
#Load the data and focus on one df specifically

t1 = time.time()

paths , class_list, sleep_night_list , sleep_type_list , participant_id_list = constants.generate_paths_and_info(blue_pebble = True)

path = paths[0]

with io.capture_output() as captured:
    data_epo = mne.read_epochs(path)
    sampling_freq =  data_epo.info['sfreq']  # find sampling frequency
    channels = data_epo.ch_names  # extract channel names
    data = data_epo._data * 1e6  # convert signal from V to uV
    
# Generate dataframe with channels as columns, samples ordered by time _____________________________________________________
channels = constants.channel_list
df_full = data_epo.to_data_frame()
df = df_full[channels].copy()

df

t2 = time.time()

t2-t1

0.4869511127471924

### How to generate stat TS from prev script

In [30]:
#1.Generate the window indices 
window_indices = eeg_stat_ts.gen_window_indices(6, df , samp_freq = 256)
#2. Calculate bpw vals per window
bpw_per_win_df = eeg_stat_ts.gen_statistic_per_window(df , window_indices , stat = 'bpw')
#3. Convert bpw per window per channel into bpw per window per region
regional_df = eeg_stat_ts.convert_chan_stats_to_region(bpw_per_win_df, constants.channel_list , constants.region_to_channel_dict)

In [47]:
regional_df.iloc[:13,:].loc['Prefrontal'].reset_index()

,index,Prefrontal
0,Delta,0.275557
1,Theta,0.246334
2,Alpha,0.154016
3,Sigma,0.073163
4,Beta,0.209167
5,Gamma,0.041763
6,TotalAbsPow,32.308073
7,window_no.,1.000000


In [46]:
regional_df.iloc[:26,:].loc['Prefrontal'].reset_index()

,index,Delta,Theta,Alpha,Sigma,Beta,Gamma,TotalAbsPow,window_no.
0,Prefrontal,0.275557,0.246334,0.154016,0.073163,0.209167,0.041763,32.308073,1.0
1,Prefrontal,0.346026,0.224388,0.123273,0.052378,0.218702,0.035233,39.100622,2.0


In [39]:
new_row = eeg_stat_ts.dataframe_to_ts_row(regional_df.iloc[:26,:], list(regional_df.columns[:-1]) )

In [41]:
pd.DataFrame(new_row)

,Delta_Central,Theta_Central,Alpha_Central,Sigma_Central,Beta_Central,Gamma_Central,TotalAbsPow_Central,Delta_Frontal,Theta_Frontal,Alpha_Frontal,...,Beta_Right Parietal,Gamma_Right Parietal,TotalAbsPow_Right Parietal,Delta_Right Temporal,Theta_Right Temporal,Alpha_Right Temporal,Sigma_Right Temporal,Beta_Right Temporal,Gamma_Right Temporal,TotalAbsPow_Right Temporal
0,0.109404,0.528768,0.139484,0.069121,0.137460,0.015763,56.478214,0.057787,0.624778,0.148776,...,0.085209,0.029426,57.729249,0.103704,0.683575,0.102381,0.037748,0.061342,0.011249,65.601458
1,0.128268,0.357538,0.247070,0.071003,0.169332,0.026789,35.491996,0.099344,0.270435,0.266792,...,0.137645,0.039926,34.627984,0.129549,0.566254,0.173220,0.047110,0.070038,0.013828,46.966847


In [ ]:
#1.Generate the window indices 
window_indices = eeg_stat_ts.gen_window_indices(6, df , samp_freq = 256)
#2. Calculate bpw vals per window
bpw_per_win_df = eeg_stat_ts.gen_statistic_per_window(df , window_indices , stat = 'bpw')
#3. Convert bpw per window per channel into bpw per window per region
regional_df = eeg_stat_ts.convert_chan_stats_to_region(bpw_per_win_df, constants.channel_list , constants.region_to_channel_dict)
#4. Convert into a single row of a new dataframe where each cell is a series
new_row = eeg_stat_ts.dataframe_to_ts_row(regional_df, list(regional_df.columns[:-1]) )
ts_row_list.append(new_row)

### Interrogating how old stat TS created

In [10]:
eeg_stat_ts.gen_band_power_vals(df, channels).head()

,Delta,Theta,Alpha,Sigma,Beta,Gamma,TotalAbsPow
Chan,,,,,,,
Fp1,0.452974,0.212308,0.105948,0.068121,0.139423,0.021225,33.246058
Fpz,0.441395,0.209596,0.110701,0.071192,0.144832,0.022284,35.400376
Fp2,0.451312,0.202308,0.107432,0.072003,0.144987,0.021958,35.748210
F1,0.262908,0.326959,0.151458,0.093678,0.124134,0.040864,37.275436
Fz,0.285050,0.326707,0.150876,0.087968,0.110853,0.038547,41.747916


In [11]:
win_s = 4
samp_freq = 256
channels = constants.channel_list

win = int(win_s * samp_freq) #Window for Welchs Periodagram
freqs, psd = welch(df.T.values , fs = samp_freq, nperseg=win, average='median')
channel_psd = pd.DataFrame(psd, index=channels , columns=freqs) #convert psd array into dataframe
channel_psd

,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,125.75,126.00,126.25,126.50,126.75,127.00,127.25,127.50,127.75,128.00
Fp1,3.956733,34.271907,22.104378,12.003463,8.205143,6.361814,4.715268,2.878589,3.267777,2.993880,...,4.118536e-11,2.269966e-11,1.094401e-11,2.872606e-12,4.366369e-12,2.111270e-11,3.172504e-11,4.542880e-11,6.099388e-11,2.089710e-11
Fpz,2.831948,35.218156,26.335248,12.278730,8.670305,6.971903,5.383386,3.134355,2.999400,2.411552,...,2.894680e-11,1.880323e-11,6.081441e-12,1.909465e-12,3.854129e-12,1.606722e-11,2.917736e-11,4.008951e-11,7.534094e-11,4.574922e-11
Fp2,6.789961,40.212369,24.199741,13.390140,9.603606,6.117441,5.530644,3.580327,3.321386,2.456535,...,3.356288e-11,2.347739e-11,7.060293e-12,2.220496e-12,5.150271e-12,2.705365e-11,3.866070e-11,5.723655e-11,8.726510e-11,5.089634e-11
F1,2.407425,17.107820,9.981516,6.309827,4.296738,3.500190,3.437879,2.216330,2.239981,2.040800,...,1.520677e-11,7.703195e-12,2.417377e-12,7.852778e-13,1.868650e-12,5.209029e-12,1.176605e-11,1.653175e-11,4.950210e-11,6.058159e-11
Fz,4.178138,27.514516,15.352950,8.127919,5.532581,4.632866,3.240429,2.646592,1.872866,2.368413,...,1.472553e-11,1.003529e-11,3.084733e-12,7.355433e-13,1.902893e-12,6.918408e-12,1.412365e-11,2.473497e-11,3.618483e-11,2.087728e-11
F2,2.883338,23.131060,14.363009,6.766560,5.497467,5.044469,3.153884,2.080999,1.706121,1.804964,...,1.181356e-11,7.879296e-12,3.583341e-12,8.694996e-13,2.126603e-12,6.596826e-12,1.295660e-11,1.777305e-11,8.540760e-11,1.193496e-10
AF7,2.183980,16.565908,11.325195,9.306425,6.962769,5.051304,3.636987,2.635438,2.090746,2.337926,...,2.134614e-11,1.335173e-11,4.329531e-12,1.165253e-12,3.350780e-12,9.409452e-12,1.944357e-11,2.477793e-11,4.874060e-11,3.212301e-11
F7,1.949647,18.342200,11.775203,8.366522,6.053058,4.482445,3.826405,2.962723,2.261670,2.569906,...,3.702852e-11,2.794190e-11,8.016396e-12,2.461207e-12,3.913786e-12,1.737463e-11,3.174261e-11,5.422212e-11,1.305876e-10,6.784838e-11
F5,1.700668,11.696553,7.348644,5.450343,4.399636,4.126774,2.690541,1.760405,1.647999,1.608109,...,2.574835e-11,1.233612e-11,4.636708e-12,1.445291e-12,3.435471e-12,8.969461e-12,1.605455e-11,2.462920e-11,8.133647e-11,5.168569e-11
F3,2.590163,15.181676,9.088449,4.261542,4.292748,4.487592,2.632915,1.981988,1.569359,1.432841,...,1.220113e-11,5.802545e-12,3.041547e-12,7.562713e-13,1.509897e-12,5.400165e-12,1.205776e-11,1.526980e-11,2.248793e-11,1.312374e-11


### Gen PSD Features (Not Series) 

In [20]:
region_to_channel_dict = constants.region_to_channel_dict

In [21]:
win_s = 4
samp_freq = 256
channels = constants.channel_list

win = int(win_s * samp_freq) #Window for Welchs Periodagram
freqs, psd = welch(df.T.values , fs = samp_freq, nperseg=win, average='median')

row_sums = np.sum(psd, axis=1, keepdims=True)
psd_normalised = np.divide(psd, row_sums)

channel_psd_normalised = pd.DataFrame(psd_normalised, index=channels , columns=freqs) #convert psd array into dataframe
region_psd_normalised = eeg_stat_ts.convert_channel_df_to_regions(channel_psd_normalised, region_to_channel_dict)

In [28]:
# Create a new DataFrame with maximum, second-highest, third-highest, and fourth-highest values
# along with their corresponding column names
sorted_values = df_region_psd_normalised.apply(lambda row: sorted(zip(row, row.index), reverse=True), axis=1)
max_values = sorted_values.apply(lambda row: row[0][0])
max_columns = sorted_values.apply(lambda row: row[0][1])
second_max_values = sorted_values.apply(lambda row: row[1][0])
second_max_columns = sorted_values.apply(lambda row: row[1][1])
third_max_values = sorted_values.apply(lambda row: row[2][0])
third_max_columns = sorted_values.apply(lambda row: row[2][1])
fourth_max_values = sorted_values.apply(lambda row: row[3][0])
fourth_max_columns = sorted_values.apply(lambda row: row[3][1])

new_df = pd.DataFrame({'Max_Value': max_values, 'Max_Column': max_columns,
                        'Second_Max_Value': second_max_values, 'Second_Max_Column': second_max_columns,
                        'Third_Max_Value': third_max_values, 'Third_Max_Column': third_max_columns,
                        'Fourth_Max_Value': fourth_max_values, 'Fourth_Max_Column': fourth_max_columns},
                       index=df_region_psd_normalised.index)
new_df

,Max_Value,Max_Column,Second_Max_Value,Second_Max_Column,Third_Max_Value,Third_Max_Column,Fourth_Max_Value,Fourth_Max_Column
Prefrontal,0.188030,0.25,0.124616,0.5,0.064626,0.75,0.045385,1.0
Frontal,0.120680,0.25,0.070925,0.5,0.037921,0.75,0.028474,7.0
Left Frontal,0.121088,0.25,0.077467,0.5,0.053832,0.75,0.042696,1.0
Right Frontal,0.143930,0.25,0.099096,0.5,0.052524,0.75,0.041915,1.0
Central,0.133940,0.25,0.085574,0.5,0.049354,0.75,0.037643,1.0
Left Central,0.127536,0.25,0.070751,0.5,0.038823,0.75,0.036652,1.0
Right Central,0.135500,0.25,0.079653,0.5,0.047498,0.75,0.031834,1.0
Left Temporal,0.102127,0.25,0.066640,0.5,0.049586,0.75,0.038011,1.0
Right Temporal,0.124278,0.25,0.081206,0.5,0.048497,0.75,0.038986,1.0
Parietal,0.124618,0.25,0.093610,0.5,0.068115,0.75,0.048307,1.0


In [34]:
new_df.columns

Index(['Max_Value', 'Max_Column', 'Second_Max_Value', 'Second_Max_Column',
       'Third_Max_Value', 'Third_Max_Column', 'Fourth_Max_Value',
       'Fourth_Max_Column'],
      dtype='object')

In [35]:
#4. Convert into a single row of a new dataframe where each cell is a series
new_row = eeg_stat_ts.dataframe_to_ts_row(new_df, list(new_df.columns) )

KeyError: 'Max_Value'